<a href="https://colab.research.google.com/github/seymatttass/Plant_Detection/blob/main/cnnSONms1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 219148673761595323
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14626652160
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16387348655427971135
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
import os
os.chdir("/content/drive/MyDrive/dataset1")

In [ ]:
# Kütüphaneleri içe aktarma
from __future__ import print_function
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras import backend as K
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
# Parametreler
batch_size = 16  # Her bir işlemde kullanılacak örnek sayısı
num_classes = 4  # Sınıf sayısı: sağlıklı, fungal, ölü, bakterili
epochs = 25  # Eğitim döngüsü sayısı
img_rows, img_cols = 224, 224  # Giriş görüntüsünün boyutları

# Girdi boyutunu belirleme (kanal düzenine göre)
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_rows, img_cols)  # Kanal ilk format
else:
    input_shape = (img_rows, img_cols, 3)  # Kanal son format

# Veri dizinlerinin varlığını kontrol etme
train_dir = "/content/drive/MyDrive/SonDeriDeriDriveSon/train"  # Eğitim veri dizini
val_dir = "/content/drive/MyDrive/SonDeriDeriDriveSon/val"  # Doğrulama veri dizini
test_dir = "/content/drive/MyDrive/SonDeriDeriDriveSon/test"  # Test veri dizini

if not os.path.exists(train_dir):
    print(f"{train_dir} dizini bulunamadı.")
if not os.path.exists(val_dir):
    print(f"{val_dir} dizini bulunamadı.")
if not os.path.exists(test_dir):
    print(f"{test_dir} dizini bulunamadı.")

# Veri artırma ve normalleştirme
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Tüm piksel değerlerini 0-1 aralığına yeniden ölçekleme
    shear_range=0.2,  # Rastgele kesme işlemi
    zoom_range=0.2,  # Rastgele yakınlaştırma
    horizontal_flip=True)  # Yatay çevirme

validation_datagen = ImageDataGenerator(rescale=1./255)  # Doğrulama verileri için sadece yeniden ölçekleme
test_datagen = ImageDataGenerator(rescale=1./255)  # Test verileri için sadece yeniden ölçekleme

# Eğitim veri yükleyicisi
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Eğitim veri dizini
    target_size=(img_rows, img_cols),  # Hedef görüntü boyutu
    batch_size=batch_size,  # Batch boyutu
    class_mode='categorical')  # Çok sınıflı sınıflandırma

# Doğrulama veri yükleyicisi
validation_generator = validation_datagen.flow_from_directory(
    val_dir,  # Doğrulama veri dizini
    target_size=(img_rows, img_cols),  # Hedef görüntü boyutu
    batch_size=batch_size,  # Batch boyutu
    class_mode='categorical')  # Çok sınıflı sınıflandırma

# Test veri yükleyicisi
test_generator = test_datagen.flow_from_directory(
    test_dir,  # Test veri dizini
    target_size=(img_rows, img_cols),  # Hedef görüntü boyutu
    batch_size=batch_size,  # Batch boyutu
    class_mode='categorical',
    shuffle = False)  # Çok sınıflı sınıflandırma

# Veri yükleyicilerinin şekillerini kontrol etme
print("train_generator shape:", train_generator[0][0].shape)  # Eğitim verisinin şekli
print("train_generator labels shape:", train_generator[0][1].shape)  # Eğitim verisi etiketlerinin şekli
print("validation_generator shape:", validation_generator[0][0].shape)  # Doğrulama verisinin şekli
print("validation_generator labels shape:", validation_generator[0][1].shape)  # Doğrulama verisi etiketlerinin şekli
print("test_generator shape:", test_generator[0][0].shape)  # Test verisinin şekli
print("test_generator labels shape:", test_generator[0][1].shape)  # Test verisi etiketlerinin şekli


Found 2451 images belonging to 4 classes.
Found 305 images belonging to 4 classes.
Found 310 images belonging to 4 classes.
train_generator shape: (16, 224, 224, 3)
train_generator labels shape: (16, 4)
validation_generator shape: (16, 224, 224, 3)
validation_generator labels shape: (16, 4)
test_generator shape: (16, 224, 224, 3)
test_generator labels shape: (16, 4)


In [ ]:

# Modeli oluşturma
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))  # 32 filtreli 3x3 boyutunda bir evrişim katmanı ekleyin
model.add(MaxPooling2D((2, 2)))  # 2x2 boyutunda bir maksimum havuzlama katmanı ekleyin
model.add(Conv2D(64, (3, 3), activation='relu'))  # 64 filtreli 3x3 boyutunda bir evrişim katmanı ekleyin
model.add(MaxPooling2D((2, 2)))  # 2x2 boyutunda bir maksimum havuzlama katmanı ekleyin
model.add(Conv2D(128, (3, 3), activation='relu'))  # 128 filtreli 3x3 boyutunda bir evrişim katmanı ekleyin
model.add(MaxPooling2D((2, 2)))  # 2x2 boyutunda bir maksimum havuzlama katmanı ekleyin
model.add(Flatten())  # Düzleştirme katmanı ekleyin
model.add(Dense(512, activation='relu'))  # 512 nöronlu tam bağlantılı katman ekleyin
model.add(Dropout(0.5))  # Overfitting'i önlemek için Dropout katmanı ekleyin
model.add(Dense(num_classes, activation='softmax'))  # Çıkış katmanı, sınıf sayısı kadar nöron ve softmax aktivasyon fonksiyonu

# Modeli derleme
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])  # Adam optimizasyonu ve çapraz entropi kaybı kullanarak derleme

# Modeli eğitme
history = model.fit(
    train_generator,  # Eğitim veri yükleyicisi
    steps_per_epoch=train_generator.samples // batch_size,  # Bir epoch tamamlamak için adım sayısı
    epochs=epochs,  # Eğitim döngüsü sayısı
    validation_data=validation_generator,  # Doğrulama veri yükleyicisi
    validation_steps=validation_generator.samples // batch_size)  # Bir doğrulama döngüsü tamamlamak için adım sayısı

# Eğitim doğruluğunu yazdırma
train_acc = history.history['accuracy']  # Eğitim doğruluğunu alın
val_acc = history.history['val_accuracy']  # Doğrulama doğruluğunu alın
print('Training accuracy:', train_acc)  # Eğitim doğruluğunu yazdırın
print('Validation accuracy:', val_acc)  # Doğrulama doğruluğunu yazdırın

Epoch 1/25
153/153 [==============================] - 1729s 11s/step - loss: 1.2661 - accuracy: 0.4275 - val_loss: 0.9749 - val_accuracy: 0.6053
Epoch 2/25
153/153 [==============================] - 43s 283ms/step - loss: 1.0587 - accuracy: 0.5441 - val_loss: 0.9447 - val_accuracy: 0.5987
Epoch 3/25
153/153 [==============================] - 42s 272ms/step - loss: 0.9605 - accuracy: 0.6021 - val_loss: 0.8269 - val_accuracy: 0.6711
Epoch 4/25
153/153 [==============================] - 43s 280ms/step - loss: 0.9048 - accuracy: 0.6222 - val_loss: 0.8064 - val_accuracy: 0.6678
Epoch 5/25
153/153 [==============================] - 40s 262ms/step - loss: 0.8203 - accuracy: 0.6772 - val_loss: 0.7339 - val_accuracy: 0.7039
Epoch 6/25
153/153 [==============================] - 41s 271ms/step - loss: 0.7682 - accuracy: 0.7031 - val_loss: 0.7527 - val_accuracy: 0.7533
Epoch 7/25
153/153 [==============================] - 44s 285ms/step - loss: 0.7336 - accuracy: 0.7170 - val_loss: 0.7371 - val_ac

In [ ]:
# Modeli değerlendirme
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)  # Test verisini değerlendirin
print('Test accuracy:', test_acc)  # Test doğruluğunu yazdırın

19/19 [==============================] - 240s 13s/step - loss: 0.4727 - accuracy: 0.8454
Test accuracy: 0.8453947305679321
